In [21]:
import pandas as pd

csv_file_path = "powerconsumption.csv"

df = pd.read_csv(csv_file_path)
print(df.head())


        Datetime  Temperature  Humidity  WindSpeed  GeneralDiffuseFlows  \
0  1/1/2017 0:00        6.559      73.8      0.083                0.051   
1  1/1/2017 0:10        6.414      74.5      0.083                0.070   
2  1/1/2017 0:20        6.313      74.5      0.080                0.062   
3  1/1/2017 0:30        6.121      75.0      0.083                0.091   
4  1/1/2017 0:40        5.921      75.7      0.081                0.048   

   DiffuseFlows  PowerConsumption_Zone1  PowerConsumption_Zone2  \
0         0.119             34055.69620             16128.87538   
1         0.085             29814.68354             19375.07599   
2         0.100             29128.10127             19006.68693   
3         0.096             28228.86076             18361.09422   
4         0.085             27335.69620             17872.34043   

   PowerConsumption_Zone3  
0             20240.96386  
1             20131.08434  
2             19668.43373  
3             18899.27711  
4     

In [22]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# ----------------------------
# 1. Load the Dataset
# ----------------------------
df = pd.read_csv("powerconsumption.csv", parse_dates=["Datetime"])
df.set_index("Datetime", inplace=True)


In [23]:
print("\nMissing values per column:")
print(df.isnull().sum())


Missing values per column:
Temperature               0
Humidity                  0
WindSpeed                 0
GeneralDiffuseFlows       0
DiffuseFlows              0
PowerConsumption_Zone1    0
PowerConsumption_Zone2    0
PowerConsumption_Zone3    0
dtype: int64


In [24]:
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[col] = np.where(df[col] < lower_bound, lower_bound, df[col])
    df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])

In [25]:
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

In [27]:
print("\nPreprocessed Data Preview:")
print(df.head())
print("\nSummary Statistics:")
print(df.describe())


Preprocessed Data Preview:
                     Temperature  Humidity  WindSpeed  GeneralDiffuseFlows  \
Datetime                                                                     
2017-01-01 00:00:00    -2.110349  0.356459  -0.798900            -0.698195   
2017-01-01 00:10:00    -2.135335  0.401712  -0.798900            -0.698121   
2017-01-01 00:20:00    -2.152738  0.401712  -0.800178            -0.698152   
2017-01-01 00:30:00    -2.185823  0.434035  -0.798900            -0.698040   
2017-01-01 00:40:00    -2.220285  0.479287  -0.799752            -0.698207   

                     DiffuseFlows  PowerConsumption_Zone1  \
Datetime                                                    
2017-01-01 00:00:00     -0.729346                0.239917   
2017-01-01 00:10:00     -0.729746               -0.354854   
2017-01-01 00:20:00     -0.729569               -0.451143   
2017-01-01 00:30:00     -0.729616               -0.577254   
2017-01-01 00:40:00     -0.729746               -0.702514  

In [31]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.stattools import adfuller

# ----------------------------
# 1. Load the Dataset and Set Datetime Index
# ----------------------------
df = pd.read_csv("powerconsumption.csv", parse_dates=["Datetime"])
df.set_index("Datetime", inplace=True)


# ----------------------------
# 2. Create Date/Time Columns
# ----------------------------
# Extract year, month, date, and hour from the Datetime index
df["Year"] = df.index.year
df["Month"] = df.index.month
df["Date"] = df.index.day
df["Hour"] = df.index.hour

# ----------------------------
# 3. Create Time-of-Day and Season Columns as Integer Codes
# ----------------------------
# Define a function to map hour to time-of-day
def get_time_of_day(hour):
    if 6 <= hour < 12:
        return 1   # Morning
    elif 12 <= hour < 18:
        return 2   # Afternoon
    else:
        return 3   # Night

df["TimeOfDay"] = df["Hour"].apply(get_time_of_day)

# Define a function to map month to season
def get_season(month):
    if month in [3, 4, 5]:
        return 1   # Spring
    elif month in [6, 7, 8]:
        return 2   # Summer
    elif month in [9, 10, 11]:
        return 3   # Autumn
    else:
        return 4   # Winter

df["Season"] = df["Month"].apply(get_season)

# Ensure these new columns are of integer type
df["Year"] = df["Year"].astype(int)
df["Month"] = df["Month"].astype(int)
df["Date"] = df["Date"].astype(int)
df["Hour"] = df["Hour"].astype(int)
df["TimeOfDay"] = df["TimeOfDay"].astype(int)
df["Season"] = df["Season"].astype(int)


# ----------------------------
# 5. Outlier Treatment (Optional)
# ----------------------------
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[col] = np.where(df[col] < lower_bound, lower_bound, df[col])
    df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])

# ----------------------------
# 6. Resample Data to Hourly Values
# ----------------------------
# Define aggregation rules:
# For Temperature, WindSpeed, Humidity, GeneralDiffuseFlow, DiffuseFlow -> use mean
# For Power Consumption columns (assuming they contain "PowerConsumption" in their names) -> use sum
# For date/time columns (Year, Month, Date, Hour, TimeOfDay, Season) -> take the first value

aggregation = {}
for col in df.columns:
    if col in ["Temperature", "WindSpeed", "Humidity", "GeneralDiffuseFlow", "DiffuseFlow"]:
        aggregation[col] = "mean"
    elif "PowerConsumption" in col:
        aggregation[col] = "sum"
    else:
        aggregation[col] = "first"

df_hourly = df.resample("H").agg(aggregation)

# Convert date/time related columns to int after resampling (if they are not already)
for col in ["Year", "Month", "Date", "Hour", "TimeOfDay", "Season"]:
    if col in df_hourly.columns:
        df_hourly[col] = df_hourly[col].astype(int)

print("\nHourly Aggregated Data Preview:")
print(df_hourly.head())
print("\nSummary Statistics:")
print(df_hourly.describe())




Hourly Aggregated Data Preview:
                     Temperature   Humidity  WindSpeed  GeneralDiffuseFlows  \
Datetime                                                                      
2017-01-01 00:00:00     6.196833  75.066667   0.081833                0.051   
2017-01-01 01:00:00     5.548833  77.583333   0.082000                0.048   
2017-01-01 02:00:00     5.054333  78.933333   0.082333                0.070   
2017-01-01 03:00:00     5.004333  77.083333   0.082833                0.066   
2017-01-01 04:00:00     5.097667  74.050000   0.082333                0.044   

                     DiffuseFlows  PowerConsumption_Zone1  \
Datetime                                                    
2017-01-01 00:00:00         0.119            175187.84810   
2017-01-01 01:00:00         0.096            147943.29114   
2017-01-01 02:00:00         0.096            132498.22784   
2017-01-01 03:00:00         0.108            124866.83544   
2017-01-01 04:00:00         0.134            12

C:\Users\jiasi\AppData\Local\Temp\ipykernel_6896\2429283852.py:88: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_hourly = df.resample("H").agg(aggregation)


In [32]:
df_hourly.to_csv("preprocessed_powerconsumption_hourly.csv", index=True)
print("\nPreprocessed data saved to 'preprocessed_powerconsumption_hourly.csv'.")


Preprocessed data saved to 'preprocessed_powerconsumption_hourly.csv'.
